In [ ]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np

In [ ]:
df = pd.read_csv("Portugal 2022-08-01 to 2023-08-01.csv", na_filter = True)

In [ ]:
df.columns

In [ ]:
df.nunique()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# ver se existem linhas duplicadas se sim apager se não não fazer nada
print ( df.duplicated().sum() ) # não existem duplicadas por isso não fazemos nada

In [ ]:
# verificar que na coluna name apenas temos um nome ou seja podemos apagar esta coluna pois não trará nada de novo para o modelo
print ( df['name'].describe() )
df.drop(['name'], inplace = True, axis = 1) # apaga-mos a coluna 'name'
df.head() #verificamos que a coluna não existe mais


In [ ]:
# ver se existem celulas vazias e se existirem temos de decidir preencher ou apagalas
sns.heatmap(df.isnull(),yticklabels= False,  cbar = False, cmap = 'viridis')

In [ ]:
# analisar a coluna preciptype e tentar chegar a uma decisão em o que fazer
print(df['preciptype'])
print('\n')
print ( df['preciptype'].describe() )
print('\n')
print ( df['preciptype'].value_counts() )

In [ ]:
# Decidimos como temos muitas entradas em branco que devemos apagar a coluna

df.drop(['preciptype'], inplace = True, axis = 1) # apaga-mos a coluna 'preciptype'

In [ ]:
# ver se existem celulas vazias e se existirem temos de decidir preencher ou apagalas
sns.heatmap(df.isnull(),yticklabels= False,  cbar = False, cmap = 'viridis')

In [ ]:
df.drop(['datetime'], inplace = True, axis = 1) # apaga-mos a coluna 'datetime' pois já temos essa informação
print (df.info()) #verificarmos que o datetime foi eliminado

In [ ]:
df['sunrise'] = pd.to_datetime(df['sunrise'])

# Criando colunas para dia, mês, ano, hora de o sol se levantar, minuto de o sol se levantar, segundo de o sol se levantar
df['Year']   = df['sunrise'].dt.year
df['Day']    = df['sunrise'].dt.day
df['Month']  = df['sunrise'].dt.month

df['hour_rise']   = df['sunrise'].dt.hour
df['minute_rise'] = df['sunrise'].dt.minute
df['second_rise'] = df['sunrise'].dt.second

df.drop(['sunrise'], inplace = True, axis = 1) # apaga-mos a coluna 'sunrise' pois já temos essa informação

print (df.info()) #verificarmos que o sunrise foi eliminado



In [ ]:
df['sunset'] = pd.to_datetime(df['sunset'])

# Criando colunas para dia, mês, ano, hora de o sol se levantar, minuto de o sol se levantar, segundo de o sol se levantar
df['hour_set']   = df['sunset'].dt.hour
df['minute_set'] = df['sunset'].dt.minute
df['second_set'] = df['sunset'].dt.second

df.drop(['sunset'], inplace = True, axis = 1) # apaga-mos a coluna 'sunset' pois já temos essa informação

print (df.info()) #verificarmos que o sunset foi eliminado

In [ ]:
df.head()

Vamos tratar dos restantes atributos object:
 
conditions, 
description, 
icon, 
stations, 

Provavelmente vamos juntar as colunas conditions e description e fazer 6 novas colunas rain_morning, rain_afternoon, cloud_moring, cloud_afternoon,clear_moring, clear_afternoon

In [ ]:
# analise do atributo conditions e transformação nele em números
print ( df['conditions'].describe() )
print('\n')
print ( df['conditions'].value_counts() )
print('\n')
print(df['conditions'])

In [ ]:
# analise do atributo conditions
print ( df['description'].describe() )
print('\n')
print ( df['description'].value_counts() )
print('\n')
print(df['description'])

In [ ]:
# analise do atributo ico

print ( df['icon'].describe() )
print('\n')
print ( df['icon'].value_counts() )
print('\n')
print(df['icon'])

In [ ]:
# Realize o groupby 
iguais = ['conditions','description', 'icon']
df_novo = df[iguais]
print(df_novo.head())

In [ ]:
# Calcular a contagem de todas as combinações de categorias de cada coluna
contagem_combinacoes = df.groupby(['conditions', 'description', 'icon']).size().reset_index(name='count')

# exibir um dataset com a contagem da intercessão das diferentes combinações de categorias
print(contagem_combinacoes)

Criar os atributos binários a partir da coluna descrição (temdo em conta o dataset triplo anterior)

In [ ]:
# Criar os atributos binários
df['rain_morning'] = (df['description'].str.contains('morning rain') |
                      #df['description'].str.contains('Clearing in the afternoon with early morning rain') |
                      #df['description'].str.contains('Partly cloudy throughout the day with morning rain') |
                      df['description'].str.contains('Partly cloudy throughout the day with rain in the morning and afternoon') |
                      df['description'].str.contains('Partly cloudy throughout the day with a chance of rain throughout the day') |
                      # eu tirava este(o de cima)
                      #df['description'].str.contains('Becoming cloudy in the afternoon with morning rain') |
                      #df['description'].str.contains('Cloudy skies throughout the day with morning rain') |
                      #df['description'].str.contains('Clear conditions throughout the day with morning rain') |
                      df['description'].str.contains('with rain clearing later') |
                      #df['description'].str.contains('Becoming cloudy in the afternoon with rain in the morning and afternoon') |
                      df['description'].str.contains('with rain')).astype(int)

In [ ]:
#df['rain_afternoon'] = (df['description'].str.contains('Clearing in the afternoon with afternoon rain') |
#                        df['description'].str.contains('Partly cloudy throughout the day with rain in the morning and afternoon') |
 #                       df['description'].str.contains('Partly cloudy throughout the day with afternoon rain')
 #                       # Adicione mais condições aqui conforme necessário
 #                      ).astype(int)
                       
#esta versao deve apanhar mais
df['rain_afternoon'] = (df['description'].str.contains('afternoon rain') |
                        df['description'].str.contains('rain in the morning and afternoon') |
                        df['description'].str.contains('Partly cloudy throughout the day with rain')
                       ).astype(int)

In [ ]:
df['cloud_morning'] = (df['description'].str.contains('Clearing in the afternoon') |
                       df['description'].str.contains('Partly cloudy throughout the day') |
                       #df['description'].str.contains('Partly cloudy throughout the day with afternoon rain.') |
                       #df['description'].str.contains('Partly cloudy throughout the day with late afternoon rain.') |
                       df['description'].str.contains('Cloudy skies throughout the day')
                      ).astype(int)

In [ ]:
df['cloud_afternoon'] = (df['description'].str.contains('Partly cloudy throughout the day') |
                         #df['description'].str.contains('Partly cloudy throughout the day with morning rain.') |
                         df['description'].str.contains('Becoming cloudy in the afternoon') |
                         #df['description'].str.contains('Becoming cloudy in the afternoon with morning rain.') |
                         df['description'].str.contains('Cloudy skies throughout the day')
                        ).astype(int)

In [ ]:
df['clear_morning'] = (df['description'].str.contains('Clear conditions throughout the day.') |
                      df['description'].str.contains('Becoming cloudy in the afternoon.')
                     ).astype(int)

In [ ]:
df['clear_afternoon'] = (df['description'].str.contains('Clear conditions throughout the day') |
                        df['description'].str.contains('Clearing in the afternoon') #|
                        #df['description'].str.contains('Clearing in the afternoon with morning rain.') |
                        #df['description'].str.contains('Clearing in the afternoon with early morning rain.') |
                        #df['description'].str.contains('Clearing in the afternoon with afternoon rain.') |
                        #df['description'].str.contains('Clear conditions throughout the day with morning rain.') |
                        #df['description'].str.contains('Partly cloudy throughout the day with rain clearing later.')
                       ).astype(int)

df.drop(['description'], inplace = True, axis = 1)
df.drop(['conditions'], inplace = True, axis = 1)
df.drop(['icon'], inplace = True, axis = 1)
print ( df.info() )

In [ ]:
# analise do atributo precip

print ( df['precip'].describe() )
print('\n')
print ( df['precip'].value_counts() )
print('\n')
print(df['precip'])

# conlcusão é util

In [ ]:
# analise do atributo precipprob
print ( df['precipprob'].describe() )
print('\n')
print ( df['precipprob'].value_counts() )
print('\n')
print(df['precipprob'])
# vemos que é um atributo que é ou 0 ou 100, para dizer se chove ou não logo como já temos as 
#colunas rain_morning and rain_afternoon concluimos que ele é inutil

In [ ]:
df.drop(['precipprob'], inplace = True, axis = 1) # apagar a coluna inutil

In [ ]:
# analise do atributo precipcover
print ( df['precipcover'].describe() )
print('\n')
print ( df['precipcover'].value_counts() )
print('\n')
print(df['precipcover'])
# é util deve ficar

In [ ]:
print (df.nunique())
print('---------------------------------------------------')
print(df.info())

Temos de alterar as colunas que têm muitos valores unicos, depois falar com o pessoal para dicutir qual deve ser o numero aceitavel de valores unicos.

Neste momento as colunas que têm de ser reconfiguradas são: winddir,solarradiation ,solarenergy, cloudcover,humidity e winddir.

In [ ]:
# analise do atributo precipcover
print ( df['cloudcover'].describe() )
print('\n')
print ( df['cloudcover'].value_counts() )
print('\n')
print(df['cloudcover'].nunique())
print('\n')
print(df['cloudcover'])

# dar bind
#df['winddir'] = df['winddir'].round().astype(int)

print('---------------------------------------------')


#print ( df['winddir'].describe() )
#print('\n')
#print ( df['winddir'].value_counts() )
#print('\n')
#print(df['winddir'].nunique())
#print('\n')
#print(df['winddir'])

In [ ]:
# analise do atributo precipcover
print ( df['windspeed'].describe() )
print('\n')
print ( df['windspeed'].value_counts() )
print('\n')
print(df['windspeed'].nunique())
print('\n')
print(df['windspeed'])

#dar bind
#df['windspeed'] = df['windspeed'].round().astype(int)
print('-----------------------------------------------------------')

#print ( df['windspeed'].describe() )
#print('\n')
#print ( df['windspeed'].value_counts() )
#print('\n')
#print(df['windspeed'].nunique())
#print('\n')
#print(df['windspeed'])

In [ ]:
# analise do atributo precipcover
print ( df['winddir'].describe() )
print('\n')
print ( df['winddir'].value_counts() )
print('\n')
print(df['winddir'].nunique())
print('\n')
print(df['winddir'])

# dar bind
#df['winddir'] = df['winddir'].round().astype(int)

#print('---------------------------------------------')


#print ( df['winddir'].describe() )
#print('\n')
#print ( df['winddir'].value_counts() )
#print('\n')
#print(df['winddir'].nunique())
#print('\n')
#print(df['winddir'])

In [ ]:
# analise do atributo stations

print ( df['stations'].describe() )
print('\n')
print ( df['stations'].value_counts() )
print('\n')
print(df['stations'])

# Criar uma lista única de todas as estações
all_stations = set(','.join(df['stations']).split(','))

# Criar colunas binárias para cada estação
for station in all_stations:
    df[station] = df['stations'].apply(lambda x: int(station in x))

# Remover a coluna original 'stations'
df = df.drop('stations', axis=1)

print('\n')
df.info()

In [ ]:
df.nunique()

In [ ]:
#alalise do atributo severerisk
print ( df['severerisk'].describe() )
print('\n')
print ( df['severerisk'].value_counts() )

In [ ]:
# Nunca nevou logo esta coluna é unutil
print ( df['snow'].describe() )
print('\n')
print ( df['snow'].value_counts() )
df.drop(['snow'], inplace = True, axis = 1)
df.info()

In [ ]:
# Nunca nevou logo esta coluna é unutil
print ( df['snowdepth'].describe() )
print('\n')
print ( df['snowdepth'].value_counts() )
df.drop(['snowdepth'], inplace = True, axis = 1)
df.info()

In [ ]:
# Esta estação reguistou sempre os dados meteriológicos de todos os dias loga esta coluna é unutil 
print ( df['08534099999'].describe() )
print('\n')
print ( df['08534099999'].value_counts() )
df.drop(['08534099999'], inplace = True, axis = 1)
df.info()

In [ ]:
# Esta estação reguistou sempre os dados meteriológicos de todos os dias loga esta coluna é unutil 
print ( df['08536099999'].describe() )
print('\n')
print ( df['08536099999'].value_counts() )
df.drop(['08536099999'], inplace = True, axis = 1)
df.info()

In [ ]:
# Esta estação reguistou sempre os dados meteriológicos de todos os dias loga esta coluna é unutil 
print ( df['LPPT'].describe() )
print('\n')
print ( df['LPPT'].value_counts() )
df.drop(['LPPT'], inplace = True, axis = 1)
df.info()

In [ ]:
df.nunique()

In [ ]:
print ( df['solarenergy'].describe() )
print('\n')
print(df['solarenergy'].value_counts())
i = 1
for v in df['solarenergy']:
    print("Valor ",i,"é ",v)
    i +=1

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslike', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslikemin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslikemin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslike', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslikemin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslike', data=df, kind='scatter')

Por observasão dos gráficos e da matriz de confusão concluimos que os atributos de feels like e a temperatura são quase maximos 
logo procedemos á sua eleminação

In [ ]:
df.drop(['feelslikemin'], inplace = True, axis = 1)
df.drop(['feelslikemax'], inplace = True, axis = 1)
df.drop(['feelslike'], inplace = True, axis = 1)

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'tempmin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'tempmax', data=df, kind='scatter')

Pela mesma razão de cima decidimos apagar

In [ ]:
df.drop(['temp'], inplace = True, axis = 1)

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'solarenergy', y= 'solarradiation', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'uvindex', y= 'solarenergy', data=df, kind='scatter')

Mais uma vez pelas mesmas razões vamos eleminar o atributo de solarradiation

In [ ]:
df.drop(['solarradiation'], inplace = True, axis = 1)

Pela observação da matriz de correlação conseguimos observar que o clear_morning e cloud_morning têm um fator de correlação de -1 ou seja quando um sobe o outro desce ou seja podemos retirar um deles o mesmo racioncinio pode ser dito para os atributos clear_afternoon e cloud_afternoon

In [ ]:
df.drop(['clear_morning'], inplace = True, axis = 1)
df.drop(['clear_afternoon'], inplace = True, axis = 1)

In [ ]:
plt.figure(figsize = (12,7) )
sns.boxplot(x = 'uvindex', y ='solarenergy', data = df, palette = 'winter')

In [ ]:
corr_matrix = df.corr()
f,ax =  plt.subplots(figsize = (40,40))
sns.heatmap(corr_matrix, vmin=-1, vmax=1, annot=True, square=True)

In [ ]:
X = df[['tempmax','tempmin',
     'dew','humidity','precip','precipcover',
     'windgust','windspeed','winddir','sealevelpressure','cloudcover','visibility',
     'uvindex','severerisk', 'moonphase', 'Year','Day','Month',
     'hour_rise','minute_rise','second_rise','hour_set','minute_set','second_set',
     'rain_morning','rain_afternoon','cloud_morning','cloud_afternoon',
     'D7126', '08532099999', '08535099999' ]]

In [ ]:
y = df['solarenergy']

In [ ]:
sns.set_style("whitegrid")
for at in X:
    sns.jointplot(x= at, y='solarenergy', data=df, kind='scatter')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

lm.fit(X_train,y_train)

In [ ]:
print(lm.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score

print ( explained_variance_score(y_test, predictions) )
print ( r2_score(y_test, predictions) )

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Defina o modelo de regressão linear como estimador
estimator = LinearRegression()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'fit_intercept': [True, False]
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
plt.scatter(y_test, grid_predictions)

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

estimator = DecisionTreeRegressor()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    
    'splitter':  ['best', 'random'],
    
    'max_depth': [5,10,15]
    
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize = (25,20))
tree.plot_tree(grid.best_estimator_)
plt.show()

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

estimator = DecisionTreeRegressor()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    
    'splitter':  ['best', 'random'],
    
    'max_depth': [2,3,4,5,6,7,8],   
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
fig = plt.figure(figsize = (25,20))
tree.plot_tree(grid.best_estimator_)
plt.show()

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, grid_predictions)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

estimator = DecisionTreeRegressor()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    
    'splitter':  ['best', 'random'],
    
    'max_depth': [2,3,4,5,6,7,8,9],
    
    'max_features': ['auto', 'sqrt', 'log2']
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
fig = plt.figure(figsize = (25,20))
tree.plot_tree(grid.best_estimator_)
plt.show()

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )
print(mean_absolute_percentage_error(y_test, grid_predictions))

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

estimator = SVR()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'kernel': [ 'rbf', 'sigmoid'],
    
    'gamma' : [1,0.1,0.01,0.001,0.0001 ],
    
    'C'    : [0.1,1,10,100,1000]
 
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )
print(mean_absolute_percentage_error(y_test, grid_predictions))

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

estimator = SVR()

# Defina os hiperparâmetros que você deseja ajustar e os valores a serem testados
param_grid = {
    'kernel': [ 'rbf', 'sigmoid'],
    
    'gamma' : [1,0.1,0.01,0.001,0.0001 ],
    
    'C'    : [0.1,1,10,100,1000]
 
}

# Crie o objeto GridSearchCV
grid = GridSearchCV(estimator, param_grid, refit=True, verbose=2, cv=10)

# Ajuste o modelo aos dados
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
print ( explained_variance_score(y_test, grid_predictions) )
print ( r2_score(y_test, grid_predictions) )
print(mean_absolute_percentage_error(y_test, grid_predictions))

print('MAE:', metrics.mean_absolute_error(y_test, grid_predictions))
print('MSE:', metrics.mean_squared_error(y_test, grid_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, grid_predictions)))